In [ ]:
import os
import math
from matplotlib import image
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

from activations import *
from helpers1 import *
from helpers2 import *
from helpers_conv import *
from helpers_fc import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [ ]:
dir_names = [
"pics/Zdrava_malina/",
"pics/Antrakoza_maline_Anthracnose/",
"pics/Bakteriozni_rak_Crown_Gall_and_Cane_Gall/",
"pics/Siva_trulez_Cane_botrytis_Botrytis_cinerea_Gray_Mold_Powdery/",
"pics/Bakteriozna_plamenjaca_Erwinia_amylovora/",
"pics/Malinina_musica_galica_Lasioptera_rubi/",
"pics/Narandzasta_rdja_Orange_Rust_Phragmidium/",
"pics/Plamenjača_korena_maline_Fitoftora_Fragariae_var_rubi_Phytophthora/",
"pics/Ljubicasta_pegavost_Sušenje_izdanaka_maline_Didymella_applanata_Spur_blight_and_Cane_blight/",
"pics/virus/Raspberry_bushy_dwarf_RBDV/",
"pics/virus/Tackasti_list_Leaf_spot/",
"pics/virus/uzrok_vasi/Mozaik_Raspberry_mosaic/",
"pics/virus/uzrok_vasi/Kovrdzanje_listova_Raspberry_leaf_curl/"
]

In [ ]:
# Get all pixel values
num_of_dirs = len(dir_names)
pixels, m = readAllPixels(num_of_dirs,dir_names)

In [ ]:
# Create dataset 'x' - store all values from 'pixels' in one vector 'x'
# Create dataset 'y' - 
# Images: 200x200x3, Data: m x dim(image)

x = np.zeros((m, pixels[0][0].shape[0],pixels[0][0].shape[1], pixels[0][0].shape[2]))
y = np.zeros((m, num_of_dirs))
c1, c2 = 0, 0

for i in range(num_of_dirs):
    for j in range(len(pixels[i])):
        x[c1] = pixels[i][j]
        y[c1][c2] = 1
        c1 = c1 + 1 
    c2 = c2 + 1

print("Number of all examples: " + str(x.shape[0]) + " images")
print("Dimension of dataset x: " + str(x.shape))
print("Dimension of dataset y: " + str(y.shape))
print()
print("x, y data type is: "+ str(type(x[0][0][0][0]))+ ", " + str(type(y[0][0])) )
print()
print("y values for pictures comming from first sub-directory:\n " + str(y[len(pixels[0])-1]))
print("y values for pictures comming from second sub-directory:\n " + str(y[len(pixels[0])]))
print("y values for pictures comming from third sub-directory:\n " + str(y[len(pixels[0])+len(pixels[1])]))
print("...")
print("y values for pictures comming from last sub-directory:\n " + str(y[m-1]))
    
# Free memory immediately (I)
pixels = None   

In [ ]:
# Split train and test data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

# Free memory immediately (II)
x, y = None, None

# Standardize data
x_train = x_train/255.
x_test = x_test/255.


In [ ]:
# Inspect data
m_train = x_train.shape[0]
m_test = x_test.shape[0]
num_px = x_train.shape[1]
depth = x_train.shape[3]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/width of each square image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", " + str(depth) + ")\n")
print ("x_train shape: " + str(x_train.shape))
print ("y_train shape: " + str(y_train.shape))
print ("x_test shape: " + str(x_test.shape))
print ("y_test shape: " + str(y_test.shape) + "\n")
print ("Number of all examples/images: m = " + str(m))

In [ ]:
# # # # # # MODEL # # # # # # 

In [ ]:
def model(X, Y, layers_dims_conv, channels, nc, layers_dims_fc,
          learning_rate_init = 0.0007, mini_batch_size = 64, beta1 = 0.9, beta2 = 0.999,  
          epsilon = 1e-8, num_epochs = 100, print_cost = True):
    costs = []                       # to keep track of the cost
    bcost = []
    t = 0                            # initializing the counter required for Adam update
    seed = 10                        # For grading purposes, so that your "random" minibatches are the same as ours
    
    hparameters1 = {"pad" : 2, "stride": 3}
    hparameters2 = {"stride" : 2, "f": 2}
    hparameters3 = {"pad" : 1, "stride": 3}
    hparameters4 = {"stride" : 2, "f": 2}   

    # Initialize parameters CONV
    parameters_conv = initialize_parameters_rand_CONV(layers_dims_conv,channels,nc)
    # Initialize parameters FC
    parameters_fc = initialize_parameters_he_FC(layers_dims_fc)
    # Gather parameters in python dict
    parameters_all = solve_all(parameters_conv, parameters_fc)
    L = len(parameters_fc) // 2                  # number of layers in the neural network
    # Initialize the optimizer
    v, s = initialize_adam(parameters_all)
    learning_param1 = learning_rate_init / 40
    train_num = X.shape[0]
    learning_param2 = int( train_num / mini_batch_size )
    for i in range(num_epochs):
            learning_rate = learning_rate_init * (1/(1+learning_param1*(i*learning_param2)))
        # Define the random minibatches. We increment the seed to reshuffle differently the dataset after each epoch
            seed = seed + 1
            minibatches = random_mini_batches(X, Y, mini_batch_size, seed)
            real = i + 1
            if print_cost:
                print ("Running Epoch %i..." %(real))
                print ("Learning rate = " + str(learning_rate))

            for minibatch in minibatches:         

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # Get parameters for CONV network, parameters_conv
                parameters_conv = solve_conv(parameters_all, layers_dims_conv, channels, nc)
                # Forward propagation CONV                             
                P2, cache_pool2, cache_pool1, cache_conv1, cache_conv2, c1, c2 = Convolutional_Forward(
                    minibatch_X, parameters_conv, hparameters1, hparameters2, hparameters3, hparameters4)

                # Create 2D input for FC nerwork, FC1
                FC1 = P2.reshape(P2.shape[0], -1).T
                
                # Get parameters for FC network, parameters_fc
                parameters_fc = solve_fc(parameters_all, layers_dims_conv)
                # Forward propagation FC
                AL, cache_fc = L_model_forward(FC1, parameters_fc)
                
                # Compute cost
                cost = cross_entropy_loss(AL, minibatch_Y.T)                
                if print_cost:
                    bcost.append(cost)

                # Backward propagation FC
                grads_fc = L_model_backward(AL, minibatch_Y.T, cache_fc, minibatch_X.T)    
                
                # Backward propagation CONV, I
                dP2 = pool_backward(P2, cache_pool2, mode = "max")
                dA2 = relu_backward(dP2, c2)
                dZ2, dW2, db2 = conv_backward(dA2, cache_conv2)
                # Backward propagation CONV, II
                dP1 = pool_backward(dZ2, cache_pool1, mode = "max")
                dA1 = relu_backward(dP1, c1)
                dZ1, dW1, db1 = conv_backward(dA1, cache_conv1)
                
                # Gather data
                parameters_all = solve_all(parameters_conv, parameters_fc)
                grads_all = solve_all_grads(grads_fc, dA1, dW1, db1, dA2, dW2, db2 )
                
                # Update parameters with Adam optimizer
                t = t + 1 # Adam counter
                parameters_all, v, s = update_parameters_with_adam(parameters_all, grads_all, v, s, t, 
                                                                   learning_rate, beta1, beta2,  epsilon)
                
            # Print the cost after every epoch & save
            if print_cost:
                print ("Cost after epoch %i: %f" %(real, cost))
                costs.append(cost)
                print("\n\n\n")

    # plot the cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('epochs')
    plt.title("Learning rate = " + str(learning_rate))
    plt.show()
    
    # plot the bcost
    plt.plot(bcost)
    plt.ylabel('batch cost')
    plt.xlabel('batch epochs')
    plt.title("Learning rate = " + str(learning_rate))
    plt.show()
        
    return parameters_all, cost, bcost

In [ ]:
layers_dims_conv = [5, 3]                    # In first conv layer dim(filter)=(fxfxc)=5x5xc1 
                                             # where c1 is a number of channels for the first filter
                                             # In second  conv layer dim(filter)=(fxfxc)=3x3xc2 
                                             # where c2 is a number of channels for the second filter
        
channels = [3, 4]                            # c1 = 3, c2 = 4
nc = [4, 8]                                  # Number of filters in first conv layer = 4, second = 8
layers_dims_fc = [200, 447, 774, 1094, 631, 315, 181, 127, 73, 36, 20, num_of_dirs]


# train 2-layer convolutional & 3-layer fully-connected model
parameters, cost, bcost = model(x_train, y_train, layers_dims_conv, channels, nc, layers_dims_fc)

In [ ]:
# Predict
print ("On the train set:")
pred_train = predict(x_train, y_train, parameters, layers_dims_conv, channels, nc)
print ("On the test set:")
pred_test = predict(x_test, y_test, parameters, layers_dims_conv, channels, nc)
